# LIDAR validation


In [2]:
from lithops.storage import Storage
import lithops
import pathlib
import shutil
import subprocess
import os
import fnmatch
import json
import laspy
import numpy as np

ModuleNotFoundError: No module named 'lithops'

In [7]:
DATA_BUCKET = 'cb-geospatial-wildfire'
INPUT_DATA_PREFIX = 'input-las-tiles/'


In [8]:
!export LITHOPS_CONFIG_FILE = "/home/aitor/Projects/lithops-github/lithops/.lithops_config"
os.environ['LITHOPS_CONFIG_FILE'] = '/home/aitor/Projects/lithops-github/lithops/.lithops_config'


Experiment parameters


In [86]:
FCC_WINDOW = 3
FCC_BREAKPOINT = 0.01


---
Upload dataset
---


In [39]:
LOCAL_INPUT_DIR = 'dataset/'


In [40]:
storage = Storage()


In [41]:
bucket_objects = storage.list_keys(bucket=DATA_BUCKET)

for file_name in os.listdir(LOCAL_INPUT_DIR):
    if file_name not in bucket_objects:
        key = os.path.join(INPUT_DATA_PREFIX, file_name)
        with open(os.path.join(LOCAL_INPUT_DIR, file_name), 'rb') as file:
            print(f'Uploading {key}...')
            data = file.read()
            storage.put_object(bucket=DATA_BUCKET, key=key, body=data)
            print('Ok!')
# storage.put_object()


Uploading input-las-tiles/PNOA_2017_CLM-CAS_748-4414_ORT-CLA-RGB.laz...
Ok!
Uploading input-las-tiles/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.laz...
Ok!
Uploading input-las-tiles/PNOA_2017_CLM-CAS_278-4470_ORT-CLA-RGB.laz...
Ok!
Uploading input-las-tiles/PNOA_2017_CLM-CAS_742-4404_ORT-CLA-RGB.laz...
Ok!
Uploading input-las-tiles/PNOA_2017_CLM-CAS_650-4434_ORT-CLA-RGB.laz...
Ok!
Uploading input-las-tiles/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.laz...
Ok!


---
Calculte DEM, DSM and CHM
---


In [94]:
def calculate_models(obj, storage):
    import pdal
    from osgeo import gdal
    from scipy import ndimage

    # Create temporary file paths
    tmp_path_prefix = '/tmp/geo/'
    if os.path.exists(tmp_path_prefix):
        shutil.rmtree(tmp_path_prefix)
    for subpath in ['dsm', 'dem', 'chm', 'aspect', 'slope', 'fcc']:
        os.makedirs(os.path.join(tmp_path_prefix, subpath), exist_ok=True)

    las_tile_filename = pathlib.Path(obj.key).name
    tile_key = pathlib.Path(obj.key).stem

    # Save obj to file
    data = obj.data_stream.read()
    input_file_path = os.path.join(tmp_path_prefix, las_tile_filename)
    with open(input_file_path, 'wb') as file:
        file.write(data)

    # DSM pipeline
    dsm_file_path = os.path.join(tmp_path_prefix, 'dsm', tile_key + '.gtiff')
    dsm_pipeline_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": f"{input_file_path}",
                "spatialreference": "EPSG:25830"
            },
            {
                "type": "filters.reprojection",
                "in_srs": "EPSG:25830",
                "out_srs": "EPSG:25830"
            },
            {
                "type": "filters.outlier",
                "method": "radius",
                "radius": 1.0,
                "min_k": 4
            },
            {
                "type": "filters.range",
                # Classification equals 2 (corresponding to noise points in LAS).
                "limits": "Classification![7:7]"
            },
            {
                "type": "filters.range",
                "limits": "returnnumber[1:1]"
            },
            {
                "type": "writers.gdal",
                "gdaldriver": "GTiff",
                "nodata": "-9999",
                "output_type": "max",
                "resolution": 1,
                "filename": f"{dsm_file_path}"
            }
        ]
    }
    dsm_pipeline_json_str = json.dumps(dsm_pipeline_json, indent=4)
    pipeline = pdal.Pipeline(dsm_pipeline_json_str)
    pipeline.validate()
    pipeline.loglevel = 8
    print('Executing DSM pipeline...')
    result = pipeline.execute()
    print(result)

    # DEM pipeline
    dem_file_path = os.path.join(tmp_path_prefix, 'dem', tile_key + '.gtiff')
    dem_pipeline_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": f"{input_file_path}",
                "spatialreference": "EPSG:25830"
            },
            {
                "type": "filters.reprojection",
                "in_srs": "EPSG:25830",
                "out_srs": "EPSG:25830"
            },
            {
                "type": "filters.assign",
                "assignment": "Classification[:]=0"
            },
            {
                "type": "filters.elm"
            },
            {
                "type": "filters.outlier",
                "method": "radius",
                "radius": 1.0,
                "min_k": 4
            },
            {

                "type": "filters.smrf",
                "ignore": "Classification[7:7]",
                "slope": 0.2,
                "window": 16,
                "threshold": 0.45,
                "scalar": 1.2
            },
            {
                "type": "filters.range",
                # Classification equals 2 (corresponding to ground in LAS).
                "limits": "Classification[2:2]",
            },
            {
                "type": "writers.gdal",
                "gdaldriver": "GTiff",
                "nodata": "-9999",
                "output_type": "max",
                "resolution": 1,
                "filename": f"{dem_file_path}"
            }
        ]
    }
    dem_pipeline_json_str = json.dumps(dem_pipeline_json, indent=4)
    pipeline = pdal.Pipeline(dem_pipeline_json_str)
    pipeline.validate()  # Check if json options are good
    pipeline.loglevel = 8
    print('Executing DEM pipeline...')
    result = pipeline.execute()
    print(result)

    # calculate CHM
    chm_file_path = os.path.join(tmp_path_prefix, 'chm', tile_key + '.tiff')
    cmd = ['gdal_calc.py', '-A', dem_file_path, '-B', dsm_file_path,
           '--calc="B-A"', '--NoDataValue=0', '--outfile', chm_file_path]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, close_fds=True)
    stdout, stderr = p.communicate()
    print(stdout, stderr)
    # assert p.returncode == 0

    # calculate aspect
    aspect_file_path = os.path.join(tmp_path_prefix, 'aspect', tile_key + '.tiff')
    cmd = ['gdaldem', 'aspect', dem_file_path, aspect_file_path, '-compute_edges']
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, close_fds=True)
    stdout, stderr = p.communicate()
    print(stdout, stderr)
    # assert p.returncode == 0

    # calculate slope
    slope_file_path = os.path.join(tmp_path_prefix, 'slope', tile_key + '.tiff')
    cmd = ['gdaldem', 'slope', dem_file_path, slope_file_path, '-compute_edges']
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, close_fds=True)
    stdout, stderr = p.communicate()
    print(stdout, stderr)
    # assert p.returncode == 0

    # calculate FCC
    in_ds = gdal.Open(dem_file_path)
    rows = in_ds.RasterYSize
    cols = in_ds.RasterXSize
    in_band = in_ds.GetRasterBand(1)
    data = in_band.ReadAsArray(0, 0, cols, rows).astype(np.float)
    data[data > FCC_BREAKPOINT] = 1
    data[data <= FCC_BREAKPOINT] = 0

    # Computing fraction on the whole raster through a moving window.
    def _compute_fraction(array):
        nveg = np.sum(array == 1)
        total = len(array)
        out = (nveg/total)*100
        return(out)

    TCC = ndimage.generic_filter(data, _compute_fraction, size=FCC_WINDOW)

    gtiff_driver = gdal.GetDriverByName("GTiff")
    fcc_file_path = os.path.join(tmp_path_prefix, 'fcc', tile_key + '.tiff')
    out_ds = gtiff_driver.Create(fcc_file_path, cols, rows, 1, in_band.DataType)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())

    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(TCC)
    # out_ds.BuildOverviews("Average", [2, 4, 8, 16, 32])
    out_ds.FlushCache()
    del in_ds, out_ds

    outputs = [dsm_file_path, dem_file_path, chm_file_path,
               aspect_file_path, slope_file_path, fcc_file_path]
    for output_path in outputs:
        if os.path.exists(output_path):
            with open(output_path, 'rb') as output_file:
                data = output_file.read()
                cos_key = output_path.replace(tmp_path_prefix, '')
                storage.put_object(bucket=DATA_BUCKET, key=cos_key, body=data)
        else:
            print(f'Failed to upload {output_path}')

    out = subprocess.check_output(['find', '/tmp/geo/'])
    return out


In [95]:
fexec = lithops.FunctionExecutor()


2022-04-19 14:18:35,810 [INFO] lithops.config -- Lithops v2.6.1.dev0
2022-04-19 14:18:35,813 [DEBUG] lithops.config -- Loading configuration from /home/aitor/Projects/lithops-github/lithops/.lithops_config
2022-04-19 14:18:35,823 [DEBUG] lithops.config -- Loading Serverless backend module: ibm_cf
2022-04-19 14:18:35,825 [DEBUG] lithops.config -- Loading Storage backend module: ibm_cos
2022-04-19 14:18:35,826 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Creating IBM COS client
2022-04-19 14:18:35,827 [DEBUG] lithops.storage.backends.ibm_cos.ibm_cos -- Set IBM COS Endpoint to https://s3.us-south.cloud-object-storage.appdomain.cloud
2022-04-19 14:18:35,828 [DEBUG] lithops.util.ibm_token_manager -- Using IBM COS API Key - Reusing Token from local cache
2022-04-19 14:18:35,832 [DEBUG] lithops.util.ibm_token_manager -- Token expiry time: 2022-04-19 14:33:31.496553+02:00 - Minutes left: 14
2022-04-19 14:18:35,837 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS client created

In [96]:
fexec.map(calculate_models, 'cos://cb-geospatial-wildfire/input-las-tiles/')


2022-04-19 14:18:36,274 [INFO] lithops.invokers -- ExecutorID 2aa976-19 | JobID M000 - Selected Runtime: aitorarjona/cloudbutton-wildfire-ibmcf:0.1 - 2048MB
2022-04-19 14:18:36,276 [DEBUG] lithops.storage.storage -- Runtime metadata found in local memory cache
2022-04-19 14:18:36,277 [DEBUG] lithops.job.job -- ExecutorID 2aa976-19 | JobID M000 - Calling map on partitions from object storage flow
2022-04-19 14:18:36,278 [DEBUG] lithops.job.partitioner -- Parsing input data
2022-04-19 14:18:36,278 [DEBUG] lithops.job.partitioner -- Chunk size and chunk number not set 
2022-04-19 14:18:36,278 [DEBUG] lithops.job.partitioner -- Listing objects in 'ibm_cos://cb-geospatial-wildfire/input-las-tiles'
2022-04-19 14:18:37,184 [DEBUG] lithops.job.partitioner -- Total objects found: 6
2022-04-19 14:18:37,185 [DEBUG] lithops.job.partitioner -- Creating 1 partitions from object input-las-tiles/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.laz (2.6MiB)
2022-04-19 14:18:37,188 [DEBUG] lithops.job.partitioner

In [97]:
res = fexec.get_result()


2022-04-19 14:18:37,665 [INFO] lithops.wait -- ExecutorID 2aa976-19 - Getting results from 6 function activations
2022-04-19 14:18:38,350 [DEBUG] lithops.invokers -- ExecutorID 2aa976-19 | JobID M000 - Calls 00000 invoked (0.777s) - Activation ID: d44f76f1ba35448a8f76f1ba35548a37
2022-04-19 14:18:38,353 [DEBUG] lithops.invokers -- ExecutorID 2aa976-19 | JobID M000 - Calls 00001 invoked (0.778s) - Activation ID: e76bf39f865c475dabf39f865c975d33
2022-04-19 14:18:38,360 [DEBUG] lithops.invokers -- ExecutorID 2aa976-19 | JobID M000 - Calls 00005 invoked (0.771s) - Activation ID: ae7ec9b2278e4983bec9b2278e39831d
2022-04-19 14:18:38,366 [DEBUG] lithops.invokers -- ExecutorID 2aa976-19 | JobID M000 - Calls 00004 invoked (0.779s) - Activation ID: 077fff2fe9b447aebfff2fe9b437ae55
2022-04-19 14:18:39,128 [DEBUG] lithops.invokers -- ExecutorID 2aa976-19 | JobID M000 - Calls 00002 invoked (1.552s) - Activation ID: 10de45d297d246a59e45d297d236a5e8
2022-04-19 14:18:39,141 [DEBUG] lithops.invokers --

In [98]:
res


[b'/tmp/geo/\n/tmp/geo/fcc\n/tmp/geo/fcc/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff\n/tmp/geo/chm\n/tmp/geo/chm/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff\n/tmp/geo/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.laz\n/tmp/geo/dem\n/tmp/geo/dem/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.gtiff\n/tmp/geo/slope\n/tmp/geo/slope/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff\n/tmp/geo/dsm\n/tmp/geo/dsm/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.gtiff\n/tmp/geo/aspect\n/tmp/geo/aspect/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff\n',
 b'/tmp/geo/\n/tmp/geo/fcc\n/tmp/geo/fcc/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.tiff\n/tmp/geo/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.laz\n/tmp/geo/chm\n/tmp/geo/chm/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.tiff\n/tmp/geo/dem\n/tmp/geo/dem/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.gtiff\n/tmp/geo/slope\n/tmp/geo/slope/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.tiff\n/tmp/geo/dsm\n/tmp/geo/dsm/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.gtiff\n/tmp/geo/aspect\n/tmp/geo/aspect/PNOA_2017_CLM-CAS_27

In [99]:
for r in res:
    print(r.decode('utf-8').strip())
    print('---')


/tmp/geo/
/tmp/geo/fcc
/tmp/geo/fcc/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff
/tmp/geo/chm
/tmp/geo/chm/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff
/tmp/geo/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.laz
/tmp/geo/dem
/tmp/geo/dem/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.gtiff
/tmp/geo/slope
/tmp/geo/slope/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff
/tmp/geo/dsm
/tmp/geo/dsm/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.gtiff
/tmp/geo/aspect
/tmp/geo/aspect/PNOA_2017_CLM-CAS_242-4422_ORT-CLA-RGB.tiff
---
/tmp/geo/
/tmp/geo/fcc
/tmp/geo/fcc/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.tiff
/tmp/geo/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.laz
/tmp/geo/chm
/tmp/geo/chm/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.tiff
/tmp/geo/dem
/tmp/geo/dem/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.gtiff
/tmp/geo/slope
/tmp/geo/slope/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.tiff
/tmp/geo/dsm
/tmp/geo/dsm/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.gtiff
/tmp/geo/aspect
/tmp/geo/aspect/PNOA_2017_CLM-CAS_274-4464_ORT-CLA-RGB.tiff
---
/tmp